In [ ]:
import copy
################################################ класс вершин ################################################
class Vertex:
    # вершина и смежные с ней вершины и их веса
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}
        self.numNeighbor = 0

    def __del__(self):
        del self.id
        del self.connectedTo
        
    # добавление смежной вершины
    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight
        self.numNeighbor += 1
    
    # получение веса с данной вершиной
    def getWeight(self, nbr):
        return self.connectedTo[nbr]
    
    # удаление вершины
    def popVertex(self, key):
        self.connectedTo.pop(key)
        self.numNeighbor -= 1
     
    # возвращает вершину если она есть в смежных
    def getVertex(self,n):
        return n in self.connectedTo

    
################################################ класс граф ################################################
class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self,key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex
            
    def __contains__(self,n):
        return n in self.vertList
    
    def addEdge(self,v,e,w=0):
        if v not in self.vertList:
            nv = self.addVertex(v)
        if e not in self.vertList:
            nv = self.addVertex(e)
        self.vertList[v].addNeighbor(e, w)
        
    def delEdge(self,v,e,w=0):
        if self.vertList[v].getVertex(e):
            self.vertList[v].popVertex(e)
            print(f"in {v} delete {e}")
        else:
            print(f"error in {v}")
    
    def delVertex(self, key):
        self.vertList.pop(key)
        self.numVertices -= 1
        print(f"Vertex: {key} is delete")
        for x in self.vertList:
            for v in range(self.vertList[x].numNeighbor): 
                if self.vertList[x].getVertex(v):
                    self.vertList[x].popVertex(v)

    def getVertices(self):
        return self.vertList.keys()
    
    
    #@classmethod
    def read(self):
        with open("graph0.txt") as text:
             for line in text:
                    lst = line.split()
                    l = len(lst)
                    lst = [int(item) for item in lst]
                    if  l == 3:
                        self.addEdge(lst[0], lst[1], lst[2])
                    elif l == 2:
                        self.addEdge(lst[0], lst[1])
                    else:
                        self.addVertex(lst[0])
    #
    def copyVert(self, new):
        for v in self.vertList:
            new[v] = copy.deepcopy(self.vertList[v])
        return new
    
    def write(self):
        with open("graph.txt", 'w') as text:
             for v in self.vertList:
                    for w in self.vertList[v].connectedTo:
                            text.write("%s %s %s" % (v, w, self.vertList[v].getWeight(w)) + '\n')
    

    
################################################ консоль ################################################
    def copyGraph(self, newGraph):
        for i in self.vertList:
            newGraph.addVertex(i)
        self.copyVert(newGraph.vertList)
        return newGraph
    
    # просмотр графа
    def lookAtGraph(self):
        for v in self.vertList:
            print("%s:" % v, end=' ')
            if self.vertList[v].numNeighbor == 0:
                    print("[]")
            else: 
                c=0
                for w in self.vertList[v].connectedTo:
                    c +=1
                    if self.vertList[v].numNeighbor == c:
                        print("[%s:%s]" % (w, self.vertList[v].getWeight(w)))
                    else: 
                        print("[%s:%s]" % (w, self.vertList[v].getWeight(w)), end =', ') 


    def console(self):
        print("Welcome to Graph!\n\nChoose an action:\n 1. Add vertex\n 2. Delete vertex\n 3. Add edge\n 4. Delete edge\n 5. View\n")
        try:
            act = int(input('Enter action:'))
            if act == 1:
                v = int(input('Add vertex:'))
                self.addVertex(v)
                print("You added vertex!")
            elif act == 2:
                v = int(input('Delete vertex:'))
                self.delVertex(v)
                print("You delete vertex!")
            elif act == 3:
                v = int(input('Add vertex:'))
                e = int(input('Add adjacent vertex:'))
                w = int(input('Add weight or 0:'))
                self.addEdge(v,e,w)
                print(f"You added edge ({v},{e}) with weight {w}")
            elif act == 4:
                v = int(input('Delete vertex:'))
                e = int(input('Delete adjacent vertex:'))
                self.delEdge(v,e,w)
                print(f"You delete edge ({v},{e})")
            elif act == 5:
                self.lookAtGraph()
            else:
                print("You entered the wrong action")
        except:
            print("You entered not a number")